In [1]:
from wetterdienst.dwd.forecasts import DWDMosmixData, DWDMosmixType
import pandas as pd
from wetterdienst.dwd.forecasts.stations import metadata_for_forecasts
import os
import numpy as np
import requests
import plotly.express as px

apiURL = "https://api.mapbox.com/directions/v5/mapbox"
apiKey = os.environ['MAPBOX_KEY']

In [13]:
def closest_point(data, point):
    p = 0.017453292519943295
    a = 0.5 - np.cos((data['LAT']-point["lat"])*p)/2 + \
              np.cos(point["lat"]*p)*np.cos(data['LAT']*p) * \
              (1-np.cos((data['LON']-point["lon"])*p)) / 2

    dist = 12742 * np.arcsin(np.sqrt(a))
    
    return data.iloc[dist.argmin()]

def get_place_address(place):
    apiURL_places = "https://api.mapbox.com/geocoding/v5/mapbox.places"

    url = "%s/%s.json?&access_token=%s" % (apiURL_places, place, apiKey)

    response = requests.get(url)
    json_data = json.loads(response.text)

    place_name = json_data['features'][0]['place_name']
    lon, lat = json_data['features'][0]['center']

    return {
        'place_name': place_name,
        'lon': lon,
        'lat': lat
    }

In [29]:
def closest_points(data, point):
    p = 0.017453292519943295
    a = 0.5 - np.cos((data['LAT']-point["lat"])*p)/2 + \
              np.cos(point["lat"]*p)*np.cos(data['LAT']*p) * \
              (1-np.cos((data['LON']-point["lon"])*p)) / 2

    dist = 12742 * np.arcsin(np.sqrt(a))
    data['dist'] = dist
    
    return data.sort_values(by='dist')

In [36]:
df = closest_points(mosmix_stations, get_place_address('Frankfurt am Main')).head()

In [55]:
df.STATION_HEIGHT

3615    156.0
2632    102.0
3608    184.0
1463    162.0
3614    125.0
Name: STATION_HEIGHT, dtype: float64

In [58]:
options = []
for _, row in df.head().iterrows():
    options.append(
        {
            "label": "%s (%2.1f,%2.1f,%3.0f m)" % (row.STATION_NAME.lower(), row.LON, row.LAT, row.STATION_HEIGHT),
            "value": row.WMO_ID
        }
    )

In [61]:
mosmix_stations[mosmix_stations.WMO_ID == ]

,WMO_ID,ICAO_ID,STATION_NAME,LAT,LON,STATION_HEIGHT,dist
0,EW002,NaN,Beveringen,53.10,12.13,71.0,408.606358
1,EW003,NaN,Calvoerde,52.25,11.19,55.0,295.000678
2,EW004,NaN,Dahlem-Berg,50.23,6.28,600.0,171.387433
3,EW005,NaN,Delitzsch,51.30,12.23,99.0,282.633234
4,EW006,NaN,Emden,53.20,7.09,0.0,360.261651
...,...,...,...,...,...,...,...
5936,P0650,GOBD,DIASS-THIES AIRPORT,14.40,-17.04,88.0,4600.598693
5937,89014,NaN,NORDENSKIOLD BASE,73.03,-13.23,495.0,2761.003628
5938,89022,NaN,HALLEY,75.27,-26.13,30.0,3206.821275
5939,89504,NaN,TROLL,72.01,2.32,1277.0,2455.505616


In [25]:
mosmix_stations[mosmix_stations['STATION_NAME'] =='FLORENZ']

,WMO_ID,ICAO_ID,STATION_NAME,LAT,LON,STATION_HEIGHT
910,16170,LIRQ,FLORENZ,43.48,11.12,40.0


In [15]:
mosmix_stations = metadata_for_forecasts()
mosmix_stations['LAT'] = mosmix_stations['LAT'].astype(float)
mosmix_stations['LON'] = mosmix_stations['LON'].astype(float)
mosmix_stations['STATION_HEIGHT'] = mosmix_stations['STATION_HEIGHT'].astype(float)

In [23]:
closest_point(mosmix_stations, get_place_address('Florence'))

WMO_ID                     16136
ICAO_ID                     LIQD
STATION_NAME      PASSO PORRETTA
LAT                        44.01
LON                        11.00
STATION_HEIGHT           1313.00
Name: 2204, dtype: object

In [26]:
get_place_address('Florence')

{'place_name': 'Firenze, Florence, Italy', 'lon': 11.25417, 'lat': 43.77139}

In [108]:
mosmix = DWDMosmixData(
    station_ids=["L648"],
    mosmix_type=DWDMosmixType.LARGE,
    humanize_parameters = True,
    tidy_data=True
)

for data in mosmix.query():
    break

# print(response.metadata)
# print(response.data)

https://opendata.dwd.de/weather/local_forecasts/mos/MOSMIX_L/single_stations/L648/kml/MOSMIX_L_LATEST_L648.kmz: 100%|██████████| 17.4k/17.4k [00:00<00:00, 74.8MiB/s]


In [109]:
df = data.data

In [113]:
df.loc[df.PARAMETER.isin(['PRECIPITATION_CONSIST_LAST_1H'])]

,STATION_ID,DATE,PARAMETER,VALUE
17290,L648,2021-01-07 10:00:00+00:00,PRECIPITATION_CONSIST_LAST_1H,0.0
17291,L648,2021-01-07 11:00:00+00:00,PRECIPITATION_CONSIST_LAST_1H,0.0
17292,L648,2021-01-07 12:00:00+00:00,PRECIPITATION_CONSIST_LAST_1H,0.0
17293,L648,2021-01-07 13:00:00+00:00,PRECIPITATION_CONSIST_LAST_1H,0.0
17294,L648,2021-01-07 14:00:00+00:00,PRECIPITATION_CONSIST_LAST_1H,0.0
...,...,...,...,...
17532,L648,2021-01-17 12:00:00+00:00,PRECIPITATION_CONSIST_LAST_1H,0.0
17533,L648,2021-01-17 13:00:00+00:00,PRECIPITATION_CONSIST_LAST_1H,0.0
17534,L648,2021-01-17 14:00:00+00:00,PRECIPITATION_CONSIST_LAST_1H,0.0
17535,L648,2021-01-17 15:00:00+00:00,PRECIPITATION_CONSIST_LAST_1H,0.0


In [114]:
def wind_components(speed, wdir):
    '''Get wind components from speed and direction.'''

    wdir = np.deg2rad(wdir)

    u = speed * np.sin(wdir)
    v = speed * np.cos(wdir)

    return u, v

In [137]:
df.loc[df.PARAMETER.isin(['WEATHER_SIGNIFICANT'])].set_index('DATE').resample('3H').nearest().reset_index()

,DATE,STATION_ID,PARAMETER,VALUE
0,2021-01-07 09:00:00+00:00,L648,WEATHER_SIGNIFICANT,3.0
1,2021-01-07 12:00:00+00:00,L648,WEATHER_SIGNIFICANT,3.0
2,2021-01-07 15:00:00+00:00,L648,WEATHER_SIGNIFICANT,3.0
3,2021-01-07 18:00:00+00:00,L648,WEATHER_SIGNIFICANT,71.0
4,2021-01-07 21:00:00+00:00,L648,WEATHER_SIGNIFICANT,71.0
...,...,...,...,...
78,2021-01-17 03:00:00+00:00,L648,WEATHER_SIGNIFICANT,3.0
79,2021-01-17 06:00:00+00:00,L648,WEATHER_SIGNIFICANT,3.0
80,2021-01-17 09:00:00+00:00,L648,WEATHER_SIGNIFICANT,3.0
81,2021-01-17 12:00:00+00:00,L648,WEATHER_SIGNIFICANT,3.0


In [124]:
u, v = wind_components(df.loc[df.PARAMETER =='WIND_SPEED', 'VALUE'].values,
                df.loc[df.PARAMETER =='WIND_DIRECTION', 'VALUE'].values)